# Content Based Filtering

## Outline
1. [Packages](#intro)<br>
2. [Dataset](#dataset)<br>
3. [Preparing the data](#prep)<br>

## 1 - Packages
### Libraries and Modules
- **NumPy (np):** NumPy is a widely-used library in Python for numerical and mathematical operations. It provides support for working with arrays and matrices, making it essential for scientific and data analysis tasks.
- **NumPy Masked Array (ma):** NumPy's Masked Array module is used for handling arrays with missing or masked data. It allows you to work with data that may contain invalid or missing values without raising errors.
- **genfromtxt:** This is a function provided by NumPy for reading data from text files and creating NumPy arrays from the data. It's particularly useful for loading data from CSV files and other text-based formats.
- **collections.defaultdict:** The defaultdict class from the collections module is a specialized dictionary-like data structure that provides default values for missing keys. It's useful for various data manipulation tasks, such as counting occurrences.
- **Pandas (pd):** Pandas is a powerful library for data manipulation and analysis. It offers data structures like DataFrames and Series, making it easier to work with structured data.
- **TensorFlow (tf):** TensorFlow is an open-source machine learning framework developed by Google. It's widely used for building and training deep learning models, including neural networks.
- **Keras:** Keras is a high-level neural networks API that runs on top of TensorFlow (or other backend engines). It provides a user-friendly interface for building and training neural networks.
- **StandardScaler:** StandardScaler is a preprocessing technique provided by scikit-learn (sklearn) for standardizing features in your data. It removes the mean and scales features to have unit variance.
- **MinMaxScaler:** MinMaxScaler, also from scikit-learn, is used to scale features to a specified range, commonly [0, 1]. It's helpful when you want to normalize your data to a specific interval.
- **train_test_split:** This function, available in scikit-learn, is used to split your dataset into separate training and testing sets. It's a crucial step in machine learning for evaluating model performance.
- **tabulate:** Tabulate is a Python library for neatly formatting and pretty-printing tabular data. It helps make data presentation more readable and user-friendly.
- **pd.set_option("display.precision", 1):** This line of code configures the Pandas library to display floating-point numbers with a precision of one decimal place by default. It affects how data is presented when you print or display Pandas DataFrames or Series.

#### Terminal Instructions:
1. go to jupyter notebook directory 
2. run 'pip install --upgrade numpy scipy' 
3. Restart Kernel if needed.

For Linux:
1. sudo pacman -S python-<import-name> // for every single import

In [1]:
import datetime
import numpy as np
import numpy.ma as ma
from numpy import genfromtxt
from collections import defaultdict
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler, PowerTransformer, Normalizer, Binarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score, f1_score
import tabulate
pd.set_option("display.precision", 1)

from IPython.display import display
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

%run preprocessing_functions.ipynb

## 2 - Dataset
Data set used is from https://grouplens.org/datasets/movielens/latest/

#### ratings.csv
- n<sub>u</sub> = 610 users
- 100,000 individual user ratings
- ratings are out of 5

#### movies.csv
- n<sub>m</sub> = 9743 movies

Below, load the raw data sets:

In [2]:
# Load the CSV file into a DataFrame (replace 'your_file.csv' with the actual file path)
df_user = pd.read_csv('/Users/RuknuddinAsrari/Documents/github/541/COMP-541-Data_mining_project/datasets/ml-latest-small/ratings.csv')
df_movie = pd.read_csv('/Users/RuknuddinAsrari/Documents/github/541/COMP-541-Data_mining_project/datasets/ml-latest-small/movies.csv')
df_links = pd.read_csv('/Users/RuknuddinAsrari/Documents/github/541/COMP-541-Data_mining_project/datasets/ml-latest-small/links.csv')
df_tags = pd.read_csv('/Users/RuknuddinAsrari/Documents/github/541/COMP-541-Data_mining_project/datasets/ml-latest-small/tags.csv')


# For linux only:
# df_user = pd.read_csv(r'./datasets/ml-latest-small/ratings.csv')
# df_movie = pd.read_csv(r'./datasets/ml-latest-small/movies.csv')
# df_links = pd.read_csv(r'./datasets/ml-latest-small/links.csv')
# df_tags = pd.read_csv(r'./datasets/ml-latest-small/tags.csv')

### Display Dataset

#### description of the df_user dataset:

- **userId:** This column represents the unique identifier for each user who has provided movie ratings. Users are numbered from 1 to 610, indicating there are 610 users in the dataset.
- **movieId:** This column represents the unique identifier for each movie that users have rated. The movieId values correspond to specific movies.
- **rating:** This column represents the rating given by a user to a particular movie. Ratings are typically on a scale, and in this dataset, ratings vary from 0.0 to 5.0. Higher values indicate a higher rating or preference for the movie.
- **timestamp:** This column represents the timestamp when the rating was recorded. It's a Unix timestamp, which is a numerical representation of a specific date and time.

The dataset contains a total of 5 rows at the beginning and 5 rows at the end as examples. Each row represents a single user's rating for a specific movie. For example, the first row indicates that User 1 gave a rating of 4.0 to the movie with movieId 1 (Toy Story(1995)), and the rating was recorded at the specified timestamp.

In [3]:
display(df_user)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


#### description of the df_movie dataset:

- **movieId:** This column represents the unique identifier for each movie in the dataset. The movieId values are used to uniquely identify each movie.
- **title:** This column contains the titles of the movies. Each row provides the title of a specific movie, along with the release year in parentheses. For example, "Toy Story (1995)" is the title of the first movie in the dataset, released in 1995.
- **genres:** This column contains information about the genres associated with each movie. Genres are typically separated by the "|" character, indicating that a movie can belong to multiple genres. For example, the first movie "Toy Story (1995)" is associated with the following genres: Adventure, Animation, Children, Comedy, and Fantasy.

The dataset contains multiple rows, each representing a different movie. In the example, there are 5 rows, each corresponding to a different movie title and associated genres.

In [4]:
display(df_movie)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


#### Description of the df_links dataset:
- **movieId:** This column represents a unique identifier for each movie in the dataset. The movieId values are used to uniquely identify each movie.
- **imdbId:** This column contains the IMDb (Internet Movie Database) identifier for each movie. IMDb is a popular online database of movies and television shows.
- **tmdbId:** This column contains the TMDb (The Movie Database) identifier for each movie. TMDb is another online database that provides information about movies, including cast, crew, and user ratings.

The dataset contains multiple rows, with each row corresponding to a different movie. For example, the first row indicates that movieId 1 corresponds to the IMDb ID 114709 and the TMDb ID 862.0.

In [5]:
display(df_links)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


#### Description of the df_tags dataset:
- **userId:** This column represents the unique identifier for each user who has tagged movies. Users are numbered, and each userId corresponds to a specific user.
- **movieId:** This column represents the unique identifier for each movie that users have tagged. The movieId values correspond to specific movies.
- **tag:** This column contains descriptive tags or keywords associated with the movies. Users can assign tags to movies based on their content, themes, or other characteristics.
- **timestamp:** This column represents the timestamp when the tagging was recorded. It's a Unix timestamp, which is a numerical representation of a specific date and time.

The dataset contains multiple rows, with each row representing a user's tag for a specific movie. For example, the first row indicates that userId 2 tagged movieId 60756 with the tags "funny," "Highly quotable," and "will ferrell" at the specified timestamp.

In [6]:
display(df_tags)

timestamp_first_row = df_tags.loc[0, 'timestamp']
date_time = datetime.datetime.utcfromtimestamp(timestamp_first_row)
print(f'user2 tagged movie60756(Step Brothers) as "funny" at: {date_time}')

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


user2 tagged movie60756(Step Brothers) as "funny" at: 2015-10-24 19:29:54


## 3- Preprocessing The Data 

### One-Hot Encode df_movies Dataset
**One-Hot Encoding** is a technique used in machine learning and data analysis to convert categorical data, such as genres in your movie dataset, into a binary matrix format. It's a way to represent categorical variables as binary vectors, where each category is represented by a binary value (0 or 1).

Here's a brief explanation and an example for the movie dataset:

The movie dataset, has a column called genres, which contains multiple movie genres separated by the "|" character. To apply one-hot encoding to this categorical data:

- Identify Unique Categories: First, you identify all unique genres present in the dataset. For example, unique genres include "Action," "Comedy," "Romance," and "Sci-Fi."
- Create Binary Columns: For each unique genre, you create a new binary column in the dataset. Each column represents one genre. If a movie belongs to a particular genre, the corresponding column gets a value of 1; otherwise, it gets a value of 0.
![df_movies_onehot](./images/movie_one_hot_encoding.jpg)

df_movie_encoded:

In [7]:
df_movie_copy = df_movie.copy()

df_movie_final, df_movie_encoded = onehot_encode_movies(df_movie_copy)

display(df_movie_encoded)
display(df_movie_final)

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9738,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9739,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9740,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,movieId,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1995,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1995,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,1995,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,1995,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,2017,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9738,193583,2017,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9739,193585,2017,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9740,193587,2018,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


 ### Create the User Profile 
1. **Collect User Ratings:** You start by gathering the ratings provided by a user for various movies. Each rating reflects how much the user liked a specific movie.
2. **Encode Movie Genres:** You have a dataset that lists the genres associated with each movie with binary classification. For each movie the user has rated, you find the genres it belongs to and mark those genres as "rated" by the user. Next, for each movie the user has rated, the genres associated with that movie will be that rating. Do this for all movies the user has rated.
3. **Calculate Genre Ratings:** For each genre, you sum up the ratings given by the user to all the movies that belong to that genre. This gives you the total rating the user has given to all movies within each genre.
4. **Average Genre Ratings:** To create a more balanced view, you calculate the average rating for each genre by dividing the total genre rating by the number of movies the user has rated within that genre. This helps account for cases where the user may have rated more or fewer movies in certain genres.
5. **Non-Scaled User Profile:** The result of these calculations is a non-scaled user profile. It represents the average ratings the user has given to different movie genres based on their past ratings. This user profile helps identify the user's preferences and can be used for various purposes, such as recommending movies of similar genres that align with their tastes.

![df_user_profile](./images/aggregate_average_user_ratings.jpg)

Aggregate and Average to get the user profile (Takes a while):

In [8]:
df_user_copy = df_user.copy()

df_aggregate_rating, df_genre_rating_count, df_user_ratings_total, df_user_final = aggregate_average_user_ratings2(df_user, df_movie_final, df_movie_encoded)

display(df_user_ratings_total)
display(df_aggregate_rating)
display(df_genre_rating_count)
display(df_user_final)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


rating     
        count mean
userId            
1         232  4.4
2          29  3.9
3          39  2.4
4         216  3.6
5          44  3.6
...       ...  ...
606      1115  3.7
607       187  3.8
608       831  3.1
609        37  3.3
610      1302  3.7

[610 rows x 2 columns]

,rating count,rating avg
userId,,
1,232,4.4
2,29,3.9
3,39,2.4
4,216,3.6
5,44,3.6
...,...,...
606,1115,3.7
607,187,3.8
608,831,3.1


,userId,rating count,rating avg
0,1,232,4.4
1,2,29,3.9
2,3,39,2.4
3,4,216,3.6
4,5,44,3.6
...,...,...,...
605,606,1115,3.7
606,607,187,3.8
607,608,831,3.1
608,609,37,3.3


,userId,rating count,rating avg
0,1,232.0,4.4
1,1,232.0,4.4
2,1,232.0,4.4
3,1,232.0,4.4
4,1,232.0,4.4
...,...,...,...
100831,610,1302.0,3.7
100832,610,1302.0,3.7
100833,610,1302.0,3.7
100834,610,1302.0,3.7


...

  0%|          | 0/100836 [00:00<?, ?it/s]

,userId,rating count,rating avg
0,1,232,4.4
1,2,29,3.9
2,3,39,2.4
3,4,216,3.6
4,5,44,3.6
...,...,...,...
605,606,1115,3.7
606,607,187,3.8
607,608,831,3.1
608,609,37,3.3


,userId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1.0,389.0,373.0,136.0,191.0,355.0,196.0,0.0,308.0,202.0,5.0,59.0,0.0,103.0,75.0,112.0,169.0,228.0,99.0,30.0
1,2.0,43.5,12.5,0.0,0.0,28.0,38.0,13.0,66.0,0.0,0.0,3.0,15.0,0.0,8.0,4.5,15.5,37.0,4.5,3.5
2,3.0,50.0,30.0,2.0,2.5,9.0,1.0,0.0,12.0,13.5,0.0,37.5,0.0,0.5,5.0,2.5,63.0,29.0,2.5,0.0
3,4.0,83.0,106.0,24.0,38.0,365.0,103.0,8.0,418.0,70.0,16.0,17.0,3.0,64.0,80.0,196.0,34.0,135.0,25.0,38.0
4,5.0,28.0,26.0,26.0,37.0,52.0,46.0,0.0,95.0,29.0,0.0,3.0,11.0,22.0,4.0,34.0,5.0,32.0,10.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,606.0,480.0,515.0,156.0,169.0,1501.0,486.0,19.0,2644.0,349.0,30.5,174.0,49.0,164.0,345.0,1328.0,281.0,701.5,246.5,58.0
606,607.0,268.0,156.0,20.0,65.0,183.0,103.0,0.0,329.0,75.0,0.0,144.0,5.0,18.0,79.0,102.0,117.0,251.0,25.0,8.0
607,608.0,922.5,583.0,171.5,216.5,971.5,527.5,18.0,962.5,333.0,15.0,322.0,48.0,91.0,245.0,306.0,550.5,916.0,68.0,29.0
608,609.0,34.0,32.0,3.0,6.0,23.0,21.0,6.0,64.0,3.0,0.0,7.0,3.0,0.0,0.0,16.0,15.0,46.0,14.0,4.0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,90,84,28,41,82,45,0,68,46,1,17,0,22,18,26,40,55,22,7
1,11,3,0,0,7,9,3,16,0,0,1,4,0,2,1,4,10,1,1
2,14,11,4,5,9,2,0,15,4,0,8,0,1,1,5,15,7,5,0
3,25,29,6,10,103,26,2,120,19,4,4,1,16,23,58,12,37,7,10
4,9,7,5,8,14,12,0,25,6,0,1,3,5,1,11,2,9,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,151,146,41,48,420,133,5,698,96,8,52,16,44,91,355,79,199,65,17
606,72,44,5,18,54,27,0,82,20,0,35,1,5,17,29,36,61,6,2
607,277,180,54,87,354,146,6,280,110,4,97,12,33,69,106,167,259,19,11
608,11,9,0,1,6,6,2,19,0,0,2,1,0,0,5,5,14,4,1


,userId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1.0,4.3,4.4,4.9,4.7,4.3,4.4,0.0,4.5,4.4,5.0,3.5,0.0,4.7,4.2,4.3,4.2,4.1,4.5,4.3
1,2.0,4.0,4.2,0.0,0.0,4.0,4.2,4.3,4.1,0.0,0.0,3.0,3.8,0.0,4.0,4.5,3.9,3.7,4.5,3.5
2,3.0,3.6,2.7,0.5,0.5,1.0,0.5,0.0,0.8,3.4,0.0,4.7,0.0,0.5,5.0,0.5,4.2,4.1,0.5,0.0
3,4.0,3.3,3.7,4.0,3.8,3.5,4.0,4.0,3.5,3.7,4.0,4.2,3.0,4.0,3.5,3.4,2.8,3.6,3.6,3.8
4,5.0,3.1,3.7,5.2,4.6,3.7,3.8,0.0,3.8,4.8,0.0,3.0,3.7,4.4,4.0,3.1,2.5,3.6,3.3,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,606.0,3.2,3.5,3.8,3.5,3.6,3.7,3.8,3.8,3.6,3.8,3.3,3.1,3.7,3.8,3.7,3.6,3.5,3.8,3.4
606,607.0,3.7,3.5,4.0,3.6,3.4,3.8,0.0,4.0,3.8,0.0,4.1,5.0,3.6,4.6,3.5,3.2,4.1,4.2,4.0
607,608.0,3.3,3.2,3.2,2.5,2.7,3.6,3.0,3.4,3.0,3.8,3.3,4.0,2.8,3.6,2.9,3.3,3.5,3.6,2.6
608,609.0,3.1,3.6,0.0,6.0,3.8,3.5,3.0,3.4,0.0,0.0,3.5,3.0,0.0,0.0,3.2,3.0,3.3,3.5,4.0


### Scaling User Profile 
![df_user_profile](./images/scaling_data.jpg)

In [9]:
# Busted in Normalization???????????????
# normalize with respect to rows????????

df_user_final_copy = df_user_final.copy()

columns_to_scale = df_user_final_copy.columns[1:] 
scalar = StandardScaler() # <-- better - MSE.RMSE are off by 1 & P/R/F1 - bad
#scalar = MinMaxScaler(feature_range=(0,5)) # <-- dogshit
#scalar = QuantileTransformer(output_distribution='normal') 
#scalar = QuantileTransformer(output_distribution='uniform')
#scalar = MaxAbsScaler()
#scalar = RobustScaler()
# scalar = PowerTransformer(method='box-cox')
# scalar = PowerTransformer(method='yeo-johnson')
# scalar = Normalizer()
# scalar = Binarizer(threshold=0.5)

# log_transformer = FunctionTransformer(np.log1p, validate=True)  # log1p is used to handle zero values

df_user_scaled = df_user_final_copy.copy() 
df_user_scaled[columns_to_scale] = scalar.fit_transform(df_user_final_copy[columns_to_scale])
# use with log_transformer
#df_user_scaled = log_transformer.fit_transform(df_user_final_copy[columns_to_scale])

pd.set_option('display.float_format', '{:.2f}'.format)
display(df_user_scaled)

,userId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1.00,1.12,1.02,0.83,0.86,1.01,0.65,-0.73,1.29,0.65,1.80,0.31,-1.60,1.10,0.42,0.86,0.83,0.68,0.65,0.98
1,2.00,0.57,0.64,-1.86,-2.31,0.52,0.50,1.56,0.57,-2.97,-0.78,-0.02,0.51,-1.61,0.28,1.11,0.40,0.02,0.65,0.55
2,3.00,-0.02,-1.34,-1.58,-1.97,-4.02,-3.77,-0.73,-5.39,-0.19,-0.78,1.18,-1.60,-1.32,1.08,-4.18,0.80,0.68,-2.08,-1.38
3,4.00,-0.40,-0.07,0.35,0.27,-0.18,0.20,1.38,-0.58,0.07,1.28,0.87,0.09,0.70,-0.13,-0.37,-0.87,-0.05,0.02,0.71
4,5.00,-0.71,0.02,1.01,0.83,0.08,0.05,-0.73,-0.02,1.01,-0.78,-0.02,0.46,0.93,0.28,-0.75,-1.28,-0.19,-0.14,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,606.00,-0.61,-0.24,0.24,0.08,-0.13,-0.15,1.28,-0.04,0.03,1.19,0.22,0.12,0.54,0.12,0.11,0.01,-0.23,0.17,0.50
606,607.00,0.21,-0.22,0.35,0.14,-0.41,0.03,-0.73,0.36,0.12,-0.78,0.77,1.21,0.47,0.80,-0.19,-0.36,0.63,0.43,0.82
607,608.00,-0.38,-0.64,-0.10,-0.62,-1.38,-0.20,0.85,-0.67,-0.48,1.15,0.20,0.65,-0.02,-0.08,-1.02,-0.31,-0.22,0.02,0.07
608,609.00,-0.74,-0.20,-1.86,1.77,0.26,-0.33,0.85,-0.79,-2.97,-0.78,0.33,0.09,-1.61,-2.91,-0.61,-0.67,-0.59,-0.03,0.82


## 3- Model
### Recommendation Matrix

In [10]:
# Finding the recommendation
df_movie_matrix = df_movie_final.copy()
df_user_matrix = df_user_scaled.copy()
display(df_movie_matrix)
display(df_user_matrix)

# Extract the 'userId' and 'movieId' column
movie_ids = df_movie_matrix['movieId']
movie_year = df_movie_matrix['year']
user_ids = df_user_matrix['userId']

# Exclude the 'userId' and 'movieId' column for matrix multiplication or other operations
df_movie_matrix = df_movie_matrix.drop(columns=['movieId'])
df_movie_matrix = df_movie_matrix.drop(columns=['year'])
df_user_matrix = df_user_matrix.drop(columns=['userId'])

df_user_matrix = df_user_matrix.T
user_item_matrix = df_movie_matrix @ df_user_matrix

# Set 'movieId' as the index of user_item_matrix
user_item_matrix['movieId'] = movie_ids
user_item_matrix.set_index('movieId', inplace=True)

user_item_matrix = user_item_matrix.T
user_item_matrix['userId'] = user_ids
user_item_matrix.set_index('userId', inplace=True)
user_item_matrix = user_item_matrix.T
user_item_matrix.columns = user_item_matrix.columns.astype(int)

#user_item_matrix.set_index('movieId', inplace=True)
#recommendation_matrix['userId'] = user_ids


display(df_movie_matrix)
display(df_user_matrix)
display(user_item_matrix)

,movieId,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1995,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1995,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,1995,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,1995,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,2017,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9738,193583,2017,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9739,193585,2017,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9740,193587,2018,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,userId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1.00,1.12,1.02,0.83,0.86,1.01,0.65,-0.73,1.29,0.65,1.80,0.31,-1.60,1.10,0.42,0.86,0.83,0.68,0.65,0.98
1,2.00,0.57,0.64,-1.86,-2.31,0.52,0.50,1.56,0.57,-2.97,-0.78,-0.02,0.51,-1.61,0.28,1.11,0.40,0.02,0.65,0.55
2,3.00,-0.02,-1.34,-1.58,-1.97,-4.02,-3.77,-0.73,-5.39,-0.19,-0.78,1.18,-1.60,-1.32,1.08,-4.18,0.80,0.68,-2.08,-1.38
3,4.00,-0.40,-0.07,0.35,0.27,-0.18,0.20,1.38,-0.58,0.07,1.28,0.87,0.09,0.70,-0.13,-0.37,-0.87,-0.05,0.02,0.71
4,5.00,-0.71,0.02,1.01,0.83,0.08,0.05,-0.73,-0.02,1.01,-0.78,-0.02,0.46,0.93,0.28,-0.75,-1.28,-0.19,-0.14,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,606.00,-0.61,-0.24,0.24,0.08,-0.13,-0.15,1.28,-0.04,0.03,1.19,0.22,0.12,0.54,0.12,0.11,0.01,-0.23,0.17,0.50
606,607.00,0.21,-0.22,0.35,0.14,-0.41,0.03,-0.73,0.36,0.12,-0.78,0.77,1.21,0.47,0.80,-0.19,-0.36,0.63,0.43,0.82
607,608.00,-0.38,-0.64,-0.10,-0.62,-1.38,-0.20,0.85,-0.67,-0.48,1.15,0.20,0.65,-0.02,-0.08,-1.02,-0.31,-0.22,0.02,0.07
608,609.00,-0.74,-0.20,-1.86,1.77,0.26,-0.33,0.85,-0.79,-2.97,-0.78,0.33,0.09,-1.61,-2.91,-0.61,-0.67,-0.59,-0.03,0.82


,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9738,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9739,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9740,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,0,1,2,3,4,5,6,7,8,9,...,600,601,602,603,604,605,606,607,608,609
Action,1.12,0.57,-0.02,-0.40,-0.71,0.04,-0.49,-0.38,-0.69,-0.12,...,1.19,-0.02,-0.61,0.07,-0.54,-0.61,0.21,-0.38,-0.74,0.03
Adventure,1.02,0.64,-1.34,-0.07,0.02,0.38,-0.45,0.27,0.13,-0.17,...,1.17,-0.53,-0.85,0.18,-0.56,-0.24,-0.22,-0.64,-0.20,0.02
Animation,0.83,-1.86,-1.58,0.35,1.01,0.39,0.16,0.90,0.35,0.28,...,0.79,0.35,-0.20,0.35,-0.17,0.24,0.35,-0.10,-1.86,0.33
Children,0.86,-2.31,-1.97,0.27,0.83,0.20,0.02,1.54,0.41,0.14,...,0.97,0.02,-0.05,0.35,-0.09,0.08,0.14,-0.62,1.77,0.22
Comedy,1.01,0.52,-4.02,-0.18,0.08,-0.44,-0.65,-0.68,0.01,-0.53,...,1.39,-0.54,-0.52,-0.46,-0.55,-0.13,-0.41,-1.38,0.26,0.12
Crime,0.65,0.50,-3.77,0.20,0.05,-0.57,-0.55,0.12,-0.74,-0.47,...,0.89,-0.08,-0.05,-0.18,-0.85,-0.15,0.03,-0.20,-0.33,0.02
Documentary,-0.73,1.56,-0.73,1.38,-0.73,-0.73,-0.73,-0.73,-0.73,-0.73,...,1.38,0.85,1.31,-0.73,0.85,1.28,-0.73,0.85,0.85,1.49
Drama,1.29,0.57,-5.39,-0.58,-0.02,-0.35,-1.21,-0.04,-0.37,-1.10,...,1.11,-0.92,-0.36,-0.58,-1.24,-0.04,0.36,-0.67,-0.79,0.12
Fantasy,0.65,-2.97,-0.19,0.07,1.01,0.06,-0.33,0.60,1.15,-0.13,...,0.97,0.60,0.02,0.10,-0.35,0.03,0.12,-0.48,-2.97,0.01
Film-Noir,1.80,-0.78,-0.78,1.28,-0.78,0.51,0.90,-0.78,1.28,-0.78,...,-0.78,1.03,1.42,-0.78,-0.78,1.19,-0.78,1.15,-0.78,1.46


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.36,-6.00,-9.11,0.45,2.96,0.59,-1.25,2.64,2.06,-0.42,...,5.29,-0.10,-1.59,0.53,-1.72,-0.02,-0.01,-3.23,-3.01,0.70
2,2.52,-4.65,-3.51,0.27,1.86,0.64,-0.76,2.41,1.69,-0.16,...,3.10,0.10,-0.87,0.63,-1.01,-0.13,0.05,-1.74,-1.41,0.25
3,1.87,1.63,-8.21,-0.55,-0.67,-0.50,-1.99,-0.90,-0.64,-0.97,...,2.34,-1.10,-0.80,-1.24,-0.99,-0.02,-0.60,-2.41,-0.35,0.22
4,3.16,2.19,-13.60,-1.13,-0.69,-0.85,-3.20,-0.93,-1.02,-2.06,...,3.45,-2.02,-1.16,-1.81,-2.22,-0.06,-0.24,-3.07,-1.14,0.33
5,1.01,0.52,-4.02,-0.18,0.08,-0.44,-0.65,-0.68,0.01,-0.53,...,1.39,-0.54,-0.52,-0.46,-0.55,-0.13,-0.41,-1.38,0.26,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,3.61,-3.75,-5.81,-0.15,1.40,0.06,-1.31,0.45,0.82,-0.51,...,4.34,0.39,-1.31,0.06,-1.61,-0.47,0.28,-2.35,-5.31,0.49
193583,2.49,-4.32,-5.80,0.24,2.11,0.02,-0.82,0.82,1.51,-0.39,...,3.15,0.41,-0.70,-0.00,-1.07,0.14,0.06,-1.96,-4.57,0.46
193585,1.29,0.57,-5.39,-0.58,-0.02,-0.35,-1.21,-0.04,-0.37,-1.10,...,1.11,-0.92,-0.36,-0.58,-1.24,-0.04,0.36,-0.67,-0.79,0.12


### Top 10 Suggested Movies

In [11]:
df_movie_reference = df_movie.copy()

# Assuming your user_item_matrix DataFrame is already set up
# Get the first 5 user columns
num_users = user_item_matrix.columns[:100]

# Initialize a dictionary to store top movies for each column
top_movies_per_column = {}

# Iterate over the first 5 user columns and get the top 10 movies
for col in num_users:
    top_movies = user_item_matrix.nlargest(10, col)[[col]]
    top_movies_per_column[col] = top_movies

# Display the top 10 movies (movieId and the relevant user column) for each of the first 5 user columns
for col, top_movies_df in top_movies_per_column.items():
    top_movie_titles = []
    top_movie_genres = []
    for movieId, value in top_movies_df.iterrows():
        top_movie_titles.append(df_movie_reference[df_movie_reference['movieId'] == movieId]['title'].values[0])
        top_movie_genres.append(df_movie_reference[df_movie_reference['movieId'] == movieId]['genres'].values[0])
    top_movies_df['title'] = top_movie_titles
    top_movies_df['genres'] = top_movie_genres
    display(top_movies_df)


userId,1,title,genres
movieId,,,
81132,9.28,Rubber (2010),Action|Adventure|Comedy|Crime|Drama|Film-Noir|...
26701,7.62,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,Action|Animation|Crime|Drama|Film-Noir|Mystery...
1907,6.95,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...
71999,6.45,Aelita: The Queen of Mars (Aelita) (1924),Action|Adventure|Drama|Fantasy|Romance|Sci-Fi|...
4719,6.44,Osmosis Jones (2001),Action|Animation|Comedy|Crime|Drama|Romance|Th...
56152,6.31,Enchanted (2007),Adventure|Animation|Children|Comedy|Fantasy|Mu...
4956,5.98,"Stunt Man, The (1980)",Action|Adventure|Comedy|Drama|Romance|Thriller
26236,5.95,"White Sun of the Desert, The (Beloe solntse pu...",Action|Adventure|Comedy|Drama|Romance|War
8481,5.94,Northwest Passage (1940),Action|Adventure|Drama|Romance|Thriller|Western


userId,2,title,genres
movieId,,,
26236,4.05,"White Sun of the Desert, The (Beloe solntse pu...",Action|Adventure|Comedy|Drama|Romance|War
70687,3.75,Paper Heart (2009),Comedy|Documentary|Drama|Romance
31617,3.53,El Cid (1961),Action|Adventure|Drama|Romance|War
77866,3.53,Robin Hood (2010),Action|Adventure|Drama|Romance|War
8481,3.45,Northwest Passage (1940),Action|Adventure|Drama|Romance|Thriller|Western
4956,3.42,"Stunt Man, The (1980)",Action|Adventure|Comedy|Drama|Romance|Thriller
459,3.40,"Getaway, The (1994)",Action|Adventure|Crime|Drama|Romance|Thriller
42015,3.40,Casanova (2005),Action|Adventure|Comedy|Drama|Romance
31367,3.35,"Chase, The (1994)",Action|Adventure|Comedy|Crime|Romance|Thriller


userId,3,title,genres
movieId,,,
2232,3.74,Cube (1997),Horror|Mystery|Sci-Fi|Thriller
7001,3.74,Invasion of the Body Snatchers (1978),Horror|Mystery|Sci-Fi|Thriller
32213,3.74,Cube Zero (2004),Horror|Mystery|Sci-Fi|Thriller
90345,3.74,"Thing, The (2011)",Horror|Mystery|Sci-Fi|Thriller
184253,3.74,The Cloverfield Paradox (2018),Horror|Mystery|Sci-Fi|Thriller
31804,3.54,Night Watch (Nochnoy dozor) (2004),Action|Fantasy|Horror|Mystery|Sci-Fi|Thriller
2454,3.07,"Fly, The (1958)",Horror|Mystery|Sci-Fi
27482,3.07,Cube 2: Hypercube (2002),Horror|Mystery|Sci-Fi
5746,3.05,Galaxy of Terror (Quest) (1981),Action|Horror|Mystery|Sci-Fi


userId,4,title,genres
movieId,,,
5735,2.25,Faces of Death (1978),Documentary|Horror
5736,2.25,Faces of Death 3 (1985),Documentary|Horror
58879,2.17,Shine a Light (2008),Documentary|Musical|IMAX
1191,2.08,Madonna: Truth or Dare (1991),Documentary|Musical
2677,2.08,Buena Vista Social Club (1999),Documentary|Musical
2859,2.08,Stop Making Sense (1984),Documentary|Musical
3142,2.08,U2: Rattle and Hum (1988),Documentary|Musical
3679,2.08,"Decline of Western Civilization, The (1981)",Documentary|Musical
5820,2.08,Standing in the Shadows of Motown (2002),Documentary|Musical


userId,5,title,genres
movieId,,,
60365,3.88,Frosty the Snowman (1969),Animation|Children|Comedy|Fantasy|Musical
661,3.81,James and the Giant Peach (1996),Adventure|Animation|Children|Fantasy|Musical
1032,3.81,Alice in Wonderland (1951),Adventure|Animation|Children|Fantasy|Musical
1881,3.81,Quest for Camelot (1998),Adventure|Animation|Children|Fantasy|Musical
73854,3.81,"Rudolph, the Red-Nosed Reindeer (1964)",Adventure|Animation|Children|Fantasy|Musical
551,3.79,"Nightmare Before Christmas, The (1993)",Animation|Children|Fantasy|Musical
596,3.79,Pinocchio (1940),Animation|Children|Fantasy|Musical
1025,3.79,"Sword in the Stone, The (1963)",Animation|Children|Fantasy|Musical
1282,3.79,Fantasia (1940),Animation|Children|Fantasy|Musical


...

userId,96,title,genres
movieId,,,
32031,8.08,Robots (2005),Adventure|Animation|Children|Comedy|Fantasy|Sc...
85261,8.06,Mars Needs Moms (2011),Action|Adventure|Animation|Children|Comedy|Sci...
26701,8.03,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,Action|Animation|Crime|Drama|Film-Noir|Mystery...
2987,7.98,Who Framed Roger Rabbit? (1988),Adventure|Animation|Children|Comedy|Crime|Fant...
81847,7.89,Tangled (2010),Animation|Children|Comedy|Fantasy|Musical|Roma...
47124,7.75,"Ant Bully, The (2006)",Adventure|Animation|Children|Comedy|Fantasy|IMAX
78499,7.75,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX
78637,7.75,Shrek Forever After (a.k.a. Shrek: The Final C...,Adventure|Animation|Children|Comedy|Fantasy|IMAX
62999,7.73,Madagascar: Escape 2 Africa (2008),Action|Adventure|Animation|Children|Comedy|IMAX


userId,97,title,genres
movieId,,,
71999,8.81,Aelita: The Queen of Mars (Aelita) (1924),Action|Adventure|Drama|Fantasy|Romance|Sci-Fi|...
4719,7.91,Osmosis Jones (2001),Action|Animation|Comedy|Crime|Drama|Romance|Th...
31367,7.89,"Chase, The (1994)",Action|Adventure|Comedy|Crime|Romance|Thriller
2367,7.41,King Kong (1976),Adventure|Fantasy|Romance|Sci-Fi|Thriller
459,7.40,"Getaway, The (1994)",Action|Adventure|Crime|Drama|Romance|Thriller
4956,7.26,"Stunt Man, The (1980)",Action|Adventure|Comedy|Drama|Romance|Thriller
2617,7.19,"Mummy, The (1999)",Action|Adventure|Comedy|Fantasy|Horror|Thriller
72165,7.19,Cirque du Freak: The Vampire's Assistant (2009),Action|Adventure|Comedy|Fantasy|Horror|Thriller
72407,7.12,"Twilight Saga: New Moon, The (2009)",Drama|Fantasy|Horror|Romance|Thriller


userId,98,title,genres
movieId,,,
27473,3.20,American Psycho II: All American Girl (2002),Comedy|Crime|Horror|Mystery|Thriller
30745,2.99,Gozu (Gokudô kyôfu dai-gekijô: Gozu) (2003),Comedy|Crime|Drama|Horror|Mystery
7835,2.97,Song of the Thin Man (1947),Comedy|Crime|Drama|Musical|Mystery|Romance
2987,2.89,Who Framed Roger Rabbit? (1988),Adventure|Animation|Children|Comedy|Crime|Fant...
81847,2.71,Tangled (2010),Animation|Children|Comedy|Fantasy|Musical|Roma...
1407,2.67,Scream (1996),Comedy|Horror|Mystery|Thriller
1717,2.67,Scream 2 (1997),Comedy|Horror|Mystery|Thriller
3273,2.67,Scream 3 (2000),Comedy|Horror|Mystery|Thriller
27584,2.67,Dead End (2003),Comedy|Horror|Mystery|Thriller


userId,99,title,genres
movieId,,,
1907,7.10,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...
364,6.62,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX
8360,6.06,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Ro...
8481,5.91,Northwest Passage (1940),Action|Adventure|Drama|Romance|Thriller|Western
108540,5.82,Ernest & Célestine (Ernest et Célestine) (2012),Adventure|Animation|Children|Comedy|Drama|Romance
26999,5.74,"Lion King II: Simba's Pride, The (1998)",Adventure|Animation|Children|Musical|Romance
48,5.68,Pocahontas (1995),Animation|Children|Drama|Musical|Romance
783,5.68,"Hunchback of Notre Dame, The (1996)",Animation|Children|Drama|Musical|Romance
4719,5.44,Osmosis Jones (2001),Action|Animation|Comedy|Crime|Drama|Romance|Th...


userId,100,title,genres
movieId,,,
81847,3.57,Tangled (2010),Animation|Children|Comedy|Fantasy|Musical|Roma...
1907,3.10,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...
595,3.09,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX
56152,3.08,Enchanted (2007),Adventure|Animation|Children|Comedy|Fantasy|Mu...
84944,3.00,Rango (2011),Action|Adventure|Animation|Children|Comedy|Wes...
85261,2.98,Mars Needs Moms (2011),Action|Adventure|Animation|Children|Comedy|Sci...
103384,2.97,"Lone Ranger, The (2013)",Action|Adventure|Western|IMAX
62999,2.95,Madagascar: Escape 2 Africa (2008),Action|Adventure|Animation|Children|Comedy|IMAX
87222,2.95,Kung Fu Panda 2 (2011),Action|Adventure|Animation|Children|Comedy|IMAX


In [12]:
# Evaluation

In [13]:
# df_predictions = user_item_matrix.copy()
# df_actual = df_user.copy()

# display(df_predictions)
# display(df_actual)

# df_actual_pivot = df_actual.pivot(index='movieId', columns='userId', values='rating')

# # Initialize a list to store scaled columns
# scaled_columns = []

# # Iterate over each user column in df_predictions
# for col in df_predictions.columns:
#     # Extract the column (user's predictions)
#     user_predictions = df_predictions[col].values.reshape(-1, 1)

#     # Find the min and max ratings for the corresponding user in df_actual_pivot
#     min_rating = df_actual_pivot[col].min()
#     max_rating = df_actual_pivot[col].max()

#     # Check if min and max ratings are the same
#     if min_rating == max_rating:
#         # If min and max are the same, either skip scaling or set a default range
#         scaled_columns.append(pd.Series(df_predictions[col], name=col))
#     else:
#         # Scale the predictions based on the user's actual min and max ratings
#         scaler = MinMaxScaler(feature_range=(min_rating, max_rating))
#         user_scaled_ratings = scaler.fit_transform(user_predictions)
#         scaled_columns.append(pd.Series(user_scaled_ratings.flatten(), name=col))

# # Concatenate all scaled columns
# df_predictions_scaled = pd.concat(scaled_columns, axis=1)

# # Display the scaled predictions DataFrame
# display(df_predictions_scaled)
# display(df_actual_pivot)

In [14]:
# # Convert df_predictions_scaled to long format
# df_predictions_long = df_predictions_scaled.reset_index().melt(id_vars='index', value_vars=df_predictions_scaled.columns)
# df_predictions_long.rename(columns={'index': 'movieId', 'variable': 'userId', 'value': 'predicted_rating'}, inplace=True)

# # Convert df_actual_pivot to long format
# df_actual_long = df_actual_pivot.reset_index().melt(id_vars='movieId', value_vars=df_actual_pivot.columns, var_name='userId', value_name='actual_rating')

# # Merge the two DataFrames on 'movieId' and 'userId'
# df_merged = pd.merge(df_actual_long, df_predictions_long, on=['movieId', 'userId'])

# # Drop NaN values (as we can't compute errors where we don't have actual ratings)
# df_merged.dropna(inplace=True)

# # Calculate MAE and RMSE
# mae = mean_absolute_error(df_merged['actual_rating'], df_merged['predicted_rating'])
# rmse = np.sqrt(mean_squared_error(df_merged['actual_rating'], df_merged['predicted_rating']))

# print(f'Mean Absolute Error: {mae}')
# print(f'Root Mean Square Error: {rmse}')


In [15]:
df_predictions = user_item_matrix.copy()
df_actual = df_user.copy()

display(df_predictions)
display(df_actual)

df_actual_pivot = df_actual.pivot(index='movieId', columns='userId', values='rating')

# Scale to min and max of actual
min_value = df_actual_pivot.min().min()
max_value = df_actual_pivot.max().max()
print(f"Minimum value in the Actual DataFrame: {min_value}")
print(f"Maximum value in the Actual DataFrame: {max_value}")

# Scale to min and max of actual
prediction_min_value = df_predictions.min().min()
prediction_max_value = df_predictions.max().max()
print(f"Minimum value in the Prediction DataFrame: {prediction_min_value}")
print(f"Maximum value in the Prediction DataFrame: {prediction_max_value}")

scaler = MinMaxScaler(feature_range=(min_value, max_value))
scaled_ratings = scaler.fit_transform(df_predictions.T).T
df_predictions_scaled = pd.DataFrame(scaled_ratings, index=df_predictions.index, columns=df_predictions.columns)

# Scale to min and max of actual
prediction_min_value = df_predictions_scaled.min().min()
prediction_max_value = df_predictions_scaled.max().max()
print(f"Minimum value in the Prediction DataFrame: {prediction_min_value}")
print(f"Maximum value in the Prediction DataFrame: {prediction_max_value}")

# scaler = MinMaxScaler(feature_range=(0, 5))
# scaled_ratings = scaler.fit_transform(df_predictions.T).T
# df_predictions_scaled = pd.DataFrame(scaled_ratings, index=df_predictions.index, columns=df_predictions.columns)

display(df_predictions_scaled)
display(df_actual_pivot)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.36,-6.00,-9.11,0.45,2.96,0.59,-1.25,2.64,2.06,-0.42,...,5.29,-0.10,-1.59,0.53,-1.72,-0.02,-0.01,-3.23,-3.01,0.70
2,2.52,-4.65,-3.51,0.27,1.86,0.64,-0.76,2.41,1.69,-0.16,...,3.10,0.10,-0.87,0.63,-1.01,-0.13,0.05,-1.74,-1.41,0.25
3,1.87,1.63,-8.21,-0.55,-0.67,-0.50,-1.99,-0.90,-0.64,-0.97,...,2.34,-1.10,-0.80,-1.24,-0.99,-0.02,-0.60,-2.41,-0.35,0.22
4,3.16,2.19,-13.60,-1.13,-0.69,-0.85,-3.20,-0.93,-1.02,-2.06,...,3.45,-2.02,-1.16,-1.81,-2.22,-0.06,-0.24,-3.07,-1.14,0.33
5,1.01,0.52,-4.02,-0.18,0.08,-0.44,-0.65,-0.68,0.01,-0.53,...,1.39,-0.54,-0.52,-0.46,-0.55,-0.13,-0.41,-1.38,0.26,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,3.61,-3.75,-5.81,-0.15,1.40,0.06,-1.31,0.45,0.82,-0.51,...,4.34,0.39,-1.31,0.06,-1.61,-0.47,0.28,-2.35,-5.31,0.49
193583,2.49,-4.32,-5.80,0.24,2.11,0.02,-0.82,0.82,1.51,-0.39,...,3.15,0.41,-0.70,-0.00,-1.07,0.14,0.06,-1.96,-4.57,0.46
193585,1.29,0.57,-5.39,-0.58,-0.02,-0.35,-1.21,-0.04,-0.37,-1.10,...,1.11,-0.92,-0.36,-0.58,-1.24,-0.04,0.36,-0.67,-0.79,0.12


,userId,movieId,rating,timestamp
0,1,1,4.00,964982703
1,1,3,4.00,964981247
2,1,6,4.00,964982224
3,1,47,5.00,964983815
4,1,50,5.00,964982931
...,...,...,...,...
100831,610,166534,4.00,1493848402
100832,610,168248,5.00,1493850091
100833,610,168250,5.00,1494273047
100834,610,168252,5.00,1493846352


Minimum value in the Actual DataFrame: 0.5
Maximum value in the Actual DataFrame: 5.0
Minimum value in the Prediction DataFrame: -27.958620793526904
Maximum value in the Prediction DataFrame: 17.740354518382937
Minimum value in the Prediction DataFrame: 0.4999999999999998
Maximum value in the Prediction DataFrame: 5.000000000000001


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,2.99,1.26,0.74,2.34,2.76,2.36,2.05,2.71,2.61,2.19,...,3.15,2.25,2.00,2.35,1.98,2.26,2.26,1.72,1.76,2.38
2,3.35,1.44,1.75,2.75,3.18,2.85,2.48,3.32,3.13,2.64,...,3.51,2.70,2.45,2.85,2.41,2.64,2.69,2.22,2.30,2.75
3,3.22,3.17,0.98,2.69,2.66,2.70,2.37,2.61,2.67,2.59,...,3.33,2.56,2.63,2.53,2.59,2.80,2.68,2.27,2.73,2.86
4,3.46,3.29,0.50,2.70,2.78,2.75,2.33,2.73,2.72,2.53,...,3.51,2.54,2.69,2.58,2.51,2.89,2.86,2.36,2.70,2.96
5,2.66,2.50,1.00,2.27,2.36,2.19,2.11,2.10,2.33,2.15,...,2.79,2.15,2.16,2.18,2.15,2.29,2.19,1.87,2.42,2.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,3.18,1.85,1.48,2.50,2.78,2.54,2.29,2.61,2.68,2.44,...,3.32,2.60,2.29,2.54,2.24,2.44,2.58,2.11,1.57,2.62
193583,2.73,1.25,0.93,2.24,2.65,2.19,2.01,2.37,2.52,2.11,...,2.88,2.28,2.04,2.19,1.96,2.22,2.20,1.76,1.19,2.29
193585,3.83,3.47,0.50,2.90,3.18,3.02,2.58,3.17,3.00,2.64,...,3.74,2.73,3.01,2.90,2.57,3.17,3.37,2.86,2.80,3.25


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.00,NaN,NaN,NaN,4.00,NaN,4.50,NaN,NaN,NaN,...,4.00,NaN,4.00,3.00,4.00,2.50,4.00,2.50,3.00,5.00
2,NaN,NaN,NaN,NaN,NaN,4.00,NaN,4.00,NaN,NaN,...,NaN,4.00,NaN,5.00,3.50,NaN,NaN,2.00,NaN,NaN
3,4.00,NaN,NaN,NaN,NaN,5.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# df_predictions_scaled_final = df_predictions_scaled.copy()
# df_actual_pivot_final = df_actual_pivot.copy()
# #df_predictions_scaled_final = df_predictions.copy()

# display(df_predictions_scaled_final)
# display(df_actual_pivot_final)


# # Flatten df_actual_pivot_final to long format
# df_actual_long = df_actual_pivot_final.reset_index().melt(id_vars=['movieId'], value_vars=df_actual_pivot_final.columns, var_name='userId', value_name='actual_rating')

# # Flatten df_predictions_scaled_final to long format
# df_predictions_long = df_predictions_scaled_final.reset_index().melt(id_vars=['movieId'], value_vars=df_predictions_scaled_final.columns, var_name='userId', value_name='predicted_rating')

# # Merge the two DataFrames on movieId and userId
# df_merged = pd.merge(df_actual_long, df_predictions_long, on=['userId', 'movieId'])

# # Drop NaN values as we cannot compute errors where we don't have actual ratings
# df_merged.dropna(inplace=True)

# # Calculate MAE
# mae = mean_absolute_error(df_merged['actual_rating'], df_merged['predicted_rating'])

# # Calculate RMSE
# rmse = np.sqrt(mean_squared_error(df_merged['actual_rating'], df_merged['predicted_rating']))

# print(f'Mean Absolute Error: {mae}')
# print(f'Root Mean Square Error: {rmse}')

In [17]:
df_predictions_scaled_final = df_predictions_scaled.copy()
df_actual_pivot_final = df_actual_pivot.copy()

# Match NaN values: Set predictions to NaN where actual ratings are NaN
df_predictions_scaled_final[df_actual_pivot_final.isna()] = np.nan

# Flatten both dataframes to long format
df_actual_long = df_actual_pivot_final.reset_index().melt(id_vars=['movieId'], value_vars=df_actual_pivot_final.columns, var_name='userId', value_name='actual_rating')
df_predictions_long = df_predictions_scaled_final.reset_index().melt(id_vars=['movieId'], value_vars=df_predictions_scaled_final.columns, var_name='userId', value_name='predicted_rating')

# Merge the two DataFrames on movieId and userId
df_merged = pd.merge(df_actual_long, df_predictions_long, on=['userId', 'movieId'])

# Drop rows where either actual rating or predicted rating is NaN
df_merged.dropna(inplace=True)

display(df_merged)

# Calculate MAE and RMSE
mae = mean_absolute_error(df_merged['actual_rating'], df_merged['predicted_rating'])
rmse = np.sqrt(mean_squared_error(df_merged['actual_rating'], df_merged['predicted_rating']))

print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Square Error: {rmse}')

,movieId,userId,actual_rating,predicted_rating
0,1,1,4.00,2.99
2,3,1,4.00,3.22
5,6,1,4.00,3.78
43,47,1,5.00,3.03
46,50,1,5.00,3.41
...,...,...,...,...
5931332,166534,610,4.00,3.51
5931359,168248,610,5.00,3.28
5931360,168250,610,5.00,3.13
5931361,168252,610,5.00,2.67


Mean Absolute Error: 0.9589697101237847
Root Mean Square Error: 1.158598426187174


In [18]:
# THRESHOLD 2.5 - separating satisfactory from unsatisfactory experiences - takes risks and provides diverse movie recommendations 

df_predictions_scaled_final = df_predictions_scaled.copy()
df_actual_pivot_final = df_actual_pivot.copy()

# Define a threshold for considering a rating as positive
threshold = 2.5

# Binarize actual and predicted ratings
df_actual_binary = (df_actual_pivot_final >= threshold).astype(int)
df_predictions_binary = (df_predictions_scaled_final >= threshold).astype(int)

# Match NaN values in predictions to NaN in actual ratings
# Wherever the actual ratings are NaN, set the corresponding predictions to NaN
df_predictions_binary[df_actual_pivot_final.isna()] = pd.NA

# Flatten the data
df_actual_binary_flat = df_actual_binary.stack(dropna=False)
df_predictions_binary_flat = df_predictions_binary.stack(dropna=False)

# Combine the actual and predicted ratings into a single DataFrame
combined_df = pd.DataFrame({
    'actual': df_actual_binary_flat,
    'predicted': df_predictions_binary_flat
})

# Drop rows where either actual or predicted rating is NaN
combined_df_filtered = combined_df.dropna()

# Calculate Precision, Recall, and F1 Score
precision = precision_score(combined_df_filtered['actual'], combined_df_filtered['predicted'])
recall = recall_score(combined_df_filtered['actual'], combined_df_filtered['predicted'])
f1 = f1_score(combined_df_filtered['actual'], combined_df_filtered['predicted'])

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Precision: 0.914673089413293
Recall: 0.7742031541694822
F1 Score: 0.8385964259352301


In [19]:
# THRESHOLD 3

df_predictions_scaled_final = df_predictions_scaled.copy()
df_actual_pivot_final = df_actual_pivot.copy()

# Define a threshold for considering a rating as positive
threshold = 3

# Binarize actual and predicted ratings
df_actual_binary = (df_actual_pivot_final >= threshold).astype(int)
df_predictions_binary = (df_predictions_scaled_final >= threshold).astype(int)

# Match NaN values in predictions to NaN in actual ratings
# Wherever the actual ratings are NaN, set the corresponding predictions to NaN
df_predictions_binary[df_actual_pivot_final.isna()] = pd.NA

# Flatten the data
df_actual_binary_flat = df_actual_binary.stack(dropna=False)
df_predictions_binary_flat = df_predictions_binary.stack(dropna=False)

# Combine the actual and predicted ratings into a single DataFrame
combined_df = pd.DataFrame({
    'actual': df_actual_binary_flat,
    'predicted': df_predictions_binary_flat
})

# Drop rows where either actual or predicted rating is NaN
combined_df_filtered = combined_df.dropna()

# Calculate Precision, Recall, and F1 Score
precision = precision_score(combined_df_filtered['actual'], combined_df_filtered['predicted'])
recall = recall_score(combined_df_filtered['actual'], combined_df_filtered['predicted'])
f1 = f1_score(combined_df_filtered['actual'], combined_df_filtered['predicted'])

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Precision: 0.9123413790583822
Recall: 0.45812898254711787
F1 Score: 0.6099658036150464
